# Optimización de Consultas

Exploraremos planes de consulta y optimizaciones para varios ejemplos, incluyendo optimizaciones lógicas y ejemplos con y sin "predicate pushdown".

Métodos

<a href="https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.DataFrame.explain.html#pyspark.sql.DataFrame.explain" target="_blank">DataFrame</a>: explain

### Optimización Lógica

El método explain(..) imprime los planes de consulta, opcionalmente formateados por un modo de explicación dado. Compara el siguiente plan lógico y plan físico, observando cómo Catalyst maneja las múltiples transformaciones de filter.

In [ ]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=ccae0ddba7045eaf894eb38e2fdb48460be58407a4e2ad7f2c6078f74befe82e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.master('local[*]').appName('query-opt').getOrCreate()
sc = SparkContext.getOrCreate()

In [ ]:
%pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

automobile = fetch_ucirepo(id=10)
df_auto = spark.createDataFrame(automobile.data.features)

In [ ]:
df_auto.show()

+-------+-----------+--------+--------+----------+-----------------+------+----+-----------+-----------+----------------+-----------+-----------+------+-----+------+----------+---------------+------------+-----------+------------+----------+---------+-----------+-----------------+
|  price|highway-mpg|city-mpg|peak-rpm|horsepower|compression-ratio|stroke|bore|fuel-system|engine-size|num-of-cylinders|engine-type|curb-weight|height|width|length|wheel-base|engine-location|drive-wheels| body-style|num-of-doors|aspiration|fuel-type|       make|normalized-losses|
+-------+-----------+--------+--------+----------+-----------------+------+----+-----------+-----------+----------------+-----------+-----------+------+-----+------+----------+---------------+------------+-----------+------------+----------+---------+-----------+-----------------+
|13495.0|         27|      21|  5000.0|     111.0|              9.0|  2.68|3.47|       mpfi|        130|               4|       dohc|       2548|  48.8| 6

In [ ]:
from pyspark.sql.functions import col

limit_fuel_system = (
    df_auto
    .filter(col('fuel-system') != '1bbl')
    .filter(col('fuel-system') != '2bbl')
    .filter(col('fuel-system') != '4bbl')
    .filter(col('fuel-system') != 'idi')
    .filter(col('fuel-system') != 'mfi')
    .filter(col('fuel-system') != 'mpfi')
    .filter(col('fuel-system') != 'spdi')
    .filter(col('fuel-system') != 'spfi')
)

limit_fuel_system.explain(True)

== Parsed Logical Plan ==
'Filter NOT ('fuel-system = spfi)
+- Filter NOT (fuel-system#8 = spdi)
   +- Filter NOT (fuel-system#8 = mpfi)
      +- Filter NOT (fuel-system#8 = mfi)
         +- Filter NOT (fuel-system#8 = idi)
            +- Filter NOT (fuel-system#8 = 4bbl)
               +- Filter NOT (fuel-system#8 = 2bbl)
                  +- Filter NOT (fuel-system#8 = 1bbl)
                     +- LogicalRDD [price#0, highway-mpg#1L, city-mpg#2L, peak-rpm#3, horsepower#4, compression-ratio#5, stroke#6, bore#7, fuel-system#8, engine-size#9L, num-of-cylinders#10L, engine-type#11, curb-weight#12L, height#13, width#14, length#15, wheel-base#16, engine-location#17, drive-wheels#18, body-style#19, num-of-doors#20, aspiration#21, fuel-type#22, make#23, normalized-losses#24], false

== Analyzed Logical Plan ==
price: double, highway-mpg: bigint, city-mpg: bigint, peak-rpm: double, horsepower: double, compression-ratio: double, stroke: double, bore: double, fuel-system: string, engine-size: 

In [ ]:
limit_fuel_system = (
    df_auto
    .filter((col('fuel-system').isNotNull()) &
    (col('fuel-system') != '1bbl') &
    (col('fuel-system') != '2bbl') &
    (col('fuel-system') != '4bbl') &
    (col('fuel-system') != 'idi') &
    (col('fuel-system') != 'mfi') &
    (col('fuel-system') != 'mpfi') &
    (col('fuel-system') != 'spdi') &
    (col('fuel-system') != 'spfi'))
)

limit_fuel_system.explain(True)

== Parsed Logical Plan ==
'Filter ((((((((isnotnull('fuel-system) AND NOT ('fuel-system = 1bbl)) AND NOT ('fuel-system = 2bbl)) AND NOT ('fuel-system = 4bbl)) AND NOT ('fuel-system = idi)) AND NOT ('fuel-system = mfi)) AND NOT ('fuel-system = mpfi)) AND NOT ('fuel-system = spdi)) AND NOT ('fuel-system = spfi))
+- LogicalRDD [price#0, highway-mpg#1L, city-mpg#2L, peak-rpm#3, horsepower#4, compression-ratio#5, stroke#6, bore#7, fuel-system#8, engine-size#9L, num-of-cylinders#10L, engine-type#11, curb-weight#12L, height#13, width#14, length#15, wheel-base#16, engine-location#17, drive-wheels#18, body-style#19, num-of-doors#20, aspiration#21, fuel-type#22, make#23, normalized-losses#24], false

== Analyzed Logical Plan ==
price: double, highway-mpg: bigint, city-mpg: bigint, peak-rpm: double, horsepower: double, compression-ratio: double, stroke: double, bore: double, fuel-system: string, engine-size: bigint, num-of-cylinders: bigint, engine-type: string, curb-weight: bigint, height: doubl

In [ ]:
limit_fuel_system = (
    df_auto
    .filter(col('fuel-system') != '1bbl')
    .filter(col('fuel-system') != '1bbl')
    .filter(col('fuel-system') != '1bbl')
    .filter(col('fuel-system') != '1bbl')
    .filter(col('fuel-system') != '1bbl')
    .filter(col('fuel-system') != '1bbl')
    .filter(col('fuel-system') != '1bbl')
    .filter(col('fuel-system') != '1bbl')
)

limit_fuel_system.explain(True)

== Parsed Logical Plan ==
'Filter NOT ('fuel-system = 1bbl)
+- Filter NOT (fuel-system#8 = 1bbl)
   +- Filter NOT (fuel-system#8 = 1bbl)
      +- Filter NOT (fuel-system#8 = 1bbl)
         +- Filter NOT (fuel-system#8 = 1bbl)
            +- Filter NOT (fuel-system#8 = 1bbl)
               +- Filter NOT (fuel-system#8 = 1bbl)
                  +- Filter NOT (fuel-system#8 = 1bbl)
                     +- LogicalRDD [price#0, highway-mpg#1L, city-mpg#2L, peak-rpm#3, horsepower#4, compression-ratio#5, stroke#6, bore#7, fuel-system#8, engine-size#9L, num-of-cylinders#10L, engine-type#11, curb-weight#12L, height#13, width#14, length#15, wheel-base#16, engine-location#17, drive-wheels#18, body-style#19, num-of-doors#20, aspiration#21, fuel-type#22, make#23, normalized-losses#24], false

== Analyzed Logical Plan ==
price: double, highway-mpg: bigint, city-mpg: bigint, peak-rpm: double, horsepower: double, compression-ratio: double, stroke: double, bore: double, fuel-system: string, engine-size

### Caché

Por defecto, los datos de un DataFrame están presentes en un clúster de Spark solo mientras se están procesando durante una consulta; no se almacenan automáticamente en el clúster después. (Spark es un motor de procesamiento de datos, no un sistema de almacenamiento de datos). Puedes solicitar explícitamente a Spark que persista un DataFrame en el clúster invocando su método **`cache`**.

Si decides cachear un DataFrame, siempre debes eliminarlo explícitamente de la caché invocando **`unpersist`** cuando ya no lo necesites.

<img src="https://files.training.databricks.com/images/icon_best_32.png" alt="Mejor Práctica"> Cachear un DataFrame puede ser apropiado si estás seguro de que usarás el mismo DataFrame varias veces, como en:

- Análisis exploratorio de datos
- Entrenamiento de modelos de aprendizaje automático

<img src="https://files.training.databricks.com/images/icon_warn_32.png" alt="Advertencia"> Aparte de esos casos de uso, **no** debes cachear DataFrames, ya que es probable que *degrades* el rendimiento de tu aplicación.

- Cachear consume recursos del clúster que podrían usarse para la ejecución de tareas
- Cachear puede evitar que Spark realice optimizaciones de consultas


In [ ]:
df_auto.cache()

df_auto.explain(True)

== Parsed Logical Plan ==
LogicalRDD [price#0, highway-mpg#1L, city-mpg#2L, peak-rpm#3, horsepower#4, compression-ratio#5, stroke#6, bore#7, fuel-system#8, engine-size#9L, num-of-cylinders#10L, engine-type#11, curb-weight#12L, height#13, width#14, length#15, wheel-base#16, engine-location#17, drive-wheels#18, body-style#19, num-of-doors#20, aspiration#21, fuel-type#22, make#23, normalized-losses#24], false

== Analyzed Logical Plan ==
price: double, highway-mpg: bigint, city-mpg: bigint, peak-rpm: double, horsepower: double, compression-ratio: double, stroke: double, bore: double, fuel-system: string, engine-size: bigint, num-of-cylinders: bigint, engine-type: string, curb-weight: bigint, height: double, width: double, length: double, wheel-base: double, engine-location: string, drive-wheels: string, body-style: string, num-of-doors: double, aspiration: string, fuel-type: string, make: string, normalized-losses: double
LogicalRDD [price#0, highway-mpg#1L, city-mpg#2L, peak-rpm#3, horse

In [ ]:
df_auto.unpersist()

DataFrame[price: double, highway-mpg: bigint, city-mpg: bigint, peak-rpm: double, horsepower: double, compression-ratio: double, stroke: double, bore: double, fuel-system: string, engine-size: bigint, num-of-cylinders: bigint, engine-type: string, curb-weight: bigint, height: double, width: double, length: double, wheel-base: double, engine-location: string, drive-wheels: string, body-style: string, num-of-doors: double, aspiration: string, fuel-type: string, make: string, normalized-losses: double]